In [2]:
#Use ggplot for styling.

#importing data
import csv
import pandas as pd
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
import DB


2017-04-07 20:59:44,547 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-04-07 20:59:44,550 INFO sqlalchemy.engine.base.Engine ()
2017-04-07 20:59:44,552 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-04-07 20:59:44,553 INFO sqlalchemy.engine.base.Engine ()
2017-04-07 20:59:44,554 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("access_log")
2017-04-07 20:59:44,555 INFO sqlalchemy.engine.base.Engine ()
2017-04-07 20:59:44,557 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("account_company")
2017-04-07 20:59:44,558 INFO sqlalchemy.engine.base.Engine ()
2017-04-07 20:59:44,559 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("account_external")
2017-04-07 20:59:44,559 INFO sqlalchemy.engine.base.Engine ()
2017-04-07 20:59:44,560 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("authorization_level")
2017-04-07 20:59:44,561 INFO sqlalchemy.engine.base.Engine ()
2017-0

In [5]:
######################################
#Get # vendors with incomplete details.
######################################


#Vendors did not yield any empty entries
vendors = []
vendor_addr = []
addr_header = None

#SQL-esque way:
#JOIN vendors and vendor address on the keys (vendor_id).
#Search rows for empty fields and increment.
#Iterate through vendors 


#Archaic approach as it took some time to read in.
with open('data/vendor.csv', "r") as input_file:
    reader = csv.reader(input_file, delimiter="|")
    next(reader) #skip header
    vendors = [row for row in reader]

    
with open('data/vendor_address.csv', "r") as input_file:
    reader = csv.reader(input_file, delimiter="|")
    addr_header = next(reader) #skip header
    vendor_addr = [row for row in reader]

counter = 0
vendor_ids = set()
inc_vendors = []

for idx,v in enumerate(vendors):
    for entry in v:
        #print(entry)
        if not entry:
            counter += 1
            vendor_ids.add(v[0])
            inc_vendors.append(v)
            break
            
print(counter)

for idx,v in enumerate(vendor_addr):
    for entry in v:
        if not entry and v[0] not in vendor_ids:
            counter += 1
            v_2 = []
            for e in v:
                if not e:
                    v_2.append("EMPTY")
                else:
                    v_2.append(e)
                    
            inc_vendors.append(v_2)
            vendor_ids.add(v[0])
            break
            
print(counter)
print(vendor_ids)
print(inc_vendors)

with open('analysis/incomplete_vendors.csv','w') as out_file:
    out_file.write("\n"+",".join(addr_header))
    for v in inc_vendors:
        out_file.write("\n"+",".join(v))

0
10
{'VDA-00547', 'VDA-00449', 'VDA-00597', 'VDA-00773', 'VDA-00772', 'VDA-00346', 'VDA-00398', 'VDA-00109', 'VDA-00323', 'VDA-00206'}
[['VDA-00547', 'VD-0483', 'Lewis, Shelly', '1', '2011-07-19 00:00:00', 'Stauffacherstrasse 60', 'EMPTY', '8004', 'Switzerland'], ['VDA-00323', 'VD-0211', 'Guthrie, Jonah', '1', '2005-05-19 00:00:00', 'Kindhausenstrasse 7', 'EMPTY', '8602', 'Switzerland'], ['VDA-00398', 'VD-0305', 'Mcclain, Elmo', '1', '2012-07-18 00:00:00', 'R??ffelstrasse 32', 'EMPTY', '8045', 'Switzerland'], ['VDA-00346', 'VD-0237', 'Webb, Sierra', '1', '2015-07-10 00:00:00', 'Lindauerstrasse 27', 'Tagelswangen', 'EMPTY', 'EMPTY'], ['VDA-00597', 'VD-0546', 'Bradford, Ulric', '1', '2014-09-01 00:00:00', 'Zugerstrasse 23', 'W??denswil', 'EMPTY', 'EMPTY'], ['VDA-00206', 'VD-0065', 'Morse, Carolyn', '1', '2015-04-09 00:00:00', 'bas du Village 13', 'Les Bois', 'EMPTY', 'EMPTY'], ['VDA-00773', 'VD-0571', 'Mcfadden, Stewart', '2', '2011-11-02 00:00:00', 'Im Baumgarten 35', 'EMPTY', '8332', 

In [18]:
########################
#Counting split payments
########################
from sqlalchemy import func

S = DB.sess
query = S.query(func.count(DB.Transactions.narrative), DB.Transactions.narrative).\
group_by(DB.Transactions.narrative).\
having(func.count(DB.Transactions.narrative) > 1)


for row in query:
    print(row)

transactions = []
trans_header = []

with open('data/transactions.csv', "r") as input_file:
    reader = csv.reader(input_file, delimiter="|")
    trans_header = next(reader) #skip header
    transactions = [row for row in reader]
    


payments = {}
split_counts = 0
split_payments = []

for idx, row in enumerate(transactions):
    invoice_note = row[7]
    if invoice_note not in payments:
        payments[invoice_note] = [row]
    else:
        payments[invoice_note].append(row)

for key,val in payments.items():
    if len(val) > 1:
        split_counts += len(val)
        split_payments.extend(val)

with open('analysis/split_payments.csv','w') as out:
    out.write(",".join(trans_header)+"\n")
    for t in split_payments:
        out.write(",".join(t)+"\n")

2017-04-07 21:35:39,952 INFO sqlalchemy.engine.base.Engine SELECT count(transactions.narrative) AS count_1, transactions.narrative AS transactions_narrative 
FROM transactions GROUP BY transactions.narrative 
HAVING count(transactions.narrative) > ?
2017-04-07 21:35:39,953 INFO sqlalchemy.engine.base.Engine (1,)
(4, 'payment for invoice # INV-253624')
(7, 'payment for invoice # INV-253625')
(10, 'payment for invoice # INV-253626')
(4, 'payment for invoice # INV-253627')
(4, 'payment for invoice # INV-253628')
(4, 'payment for invoice # INV-253629')
(4, 'payment for invoice # INV-253630')


In [21]:
#Consecutive input ids

def generate_next_ID(current_ID):
    number = int(current_ID.split("-")[1])
    next_number = str(number+1).zfill(9) #Padding as ID length is 9.
    return "PUO-"+next_number


def generate_next_tampered_ID(current_ID):
    number = int(current_ID.split("-")[1])
    next_number = str(number+1) #No padding even though ID length is 9.
    return "PUO-"+next_number

def generate_previous_ID(current_ID):
    number = int(current_ID.split("-")[1])
    next_number = str(number-1).zfill(9)
    return "PUO-"+next_number


def generate_previous_tampered_ID(current_ID):
    number = int(current_ID.split("-")[1])
    next_number = str(number-1)
    return "PUO-"+next_number

S = DB.sess
query = S.query(DB.InvoicePurchaseOrder)
purchase_IDs = {}

for row in query:
    purchase_IDs[row.po_id] = row.vendor_id

consecutives = 0
consec = set()

for po_ID in purchase_IDs:
    next_ID = generate_next_ID(po_ID)
    previous_ID = generate_previous_ID(po_ID)
    next_tampered_ID = generate_next_tampered_ID(po_ID)
    previous_tampered_ID = generate_previous_tampered_ID(po_ID)
    
    if next_ID in purchase_IDs:# and previous_ID in purchase_IDs:
        consec.add(next_ID)
        consec.add(po_ID)
        #consec.add(previous_ID)
    
    if next_tampered_ID in purchase_IDs:# and previous_tampered_ID in purchase_IDs:
        consec.add(next_tampered_ID)
        consec.add(po_ID)


2017-04-07 19:16:05,973 INFO sqlalchemy.engine.base.Engine SELECT invoice_purchase_order.invoice_id AS invoice_purchase_order_invoice_id, invoice_purchase_order.po_id AS invoice_purchase_order_po_id, invoice_purchase_order.vendor_id AS invoice_purchase_order_vendor_id, invoice_purchase_order.invoice_dt AS invoice_purchase_order_invoice_dt, invoice_purchase_order.invoice_image_path AS invoice_purchase_order_invoice_image_path, invoice_purchase_order.payment_due_by_dt AS invoice_purchase_order_payment_due_by_dt, invoice_purchase_order.received_dt AS invoice_purchase_order_received_dt, invoice_purchase_order.received_by_user_id AS invoice_purchase_order_received_by_user_id, invoice_purchase_order.total AS invoice_purchase_order_total 
FROM invoice_purchase_order
2017-04-07 19:16:05,975 INFO sqlalchemy.engine.base.Engine ()


TypeError: must be str, not int

In [29]:
def generate_next_ID(current_ID):
    return current_ID + 1


S = DB.sess
query = S.query(DB.InvoicePurchaseOrder)
purchase_IDs = {}
purch = []

for row in query:
    raw_ID = int(row.po_id.split("-")[1])
    purch.append(raw_ID)
    purchase_IDs[raw_ID] = row.vendor_id

purch = sorted(purch)

consecutives = 0
consec = set()

for po_ID in purchase_IDs:
    next_ID = po_ID + 1
    previous_ID = po_ID -1
    
    if next_ID in purchase_IDs:#and previous_ID in purchase_IDs:
        consec.add(po_ID)
        consec.add(next_ID)
        #onsec.add(previous_ID)
    
    if previous_ID in purchase_IDs:
        consec.add(po_ID)
        consec.add(previous_ID)
    
#print(sorted(consec))
#print(len(consec))


cons = sorted(consec)
counter = 0
for idx,element in enumerate(cons[:len(cons)-1]):
    print(cons[idx]+1,cons[idx+1],purchase_IDs[cons[idx]],purchase_IDs[cons[idx+1]])
    if cons[idx]+1 == cons[idx+1] and purchase_IDs[cons[idx]] == purchase_IDs[cons[idx+1]]:
        counter +=1
        print(cons[idx])
print(counter)

#Is 73. Should be 75. Check edge cases.



#print(sorted(consec))
#print(len(consec))


2017-04-07 19:20:46,820 INFO sqlalchemy.engine.base.Engine SELECT invoice_purchase_order.invoice_id AS invoice_purchase_order_invoice_id, invoice_purchase_order.po_id AS invoice_purchase_order_po_id, invoice_purchase_order.vendor_id AS invoice_purchase_order_vendor_id, invoice_purchase_order.invoice_dt AS invoice_purchase_order_invoice_dt, invoice_purchase_order.invoice_image_path AS invoice_purchase_order_invoice_image_path, invoice_purchase_order.payment_due_by_dt AS invoice_purchase_order_payment_due_by_dt, invoice_purchase_order.received_dt AS invoice_purchase_order_received_dt, invoice_purchase_order.received_by_user_id AS invoice_purchase_order_received_by_user_id, invoice_purchase_order.total AS invoice_purchase_order_total 
FROM invoice_purchase_order
2017-04-07 19:20:46,821 INFO sqlalchemy.engine.base.Engine ()
709772 709772 VD-0449 VD-0449
709771
709773 709773 VD-0449 VD-0572
709774 709774 VD-0572 VD-0359
709775 709775 VD-0359 VD-0076
709776 709776 VD-0076 VD-0449
709777 7097

In [157]:
print("PUO-001887914" in consec)
print("PUO-001546994" in consec)

False
False


In [6]:
#Amount Stolen
from datetime import datetime
S = DB.sess
query = S.query(DB.Transactions).filter_by(approved_by_user_id="u8284") #account used for fraud.

amount = 0
accounts = {}
for row in query:
    amount += float(row.amount)
    accounts[row.credit_acct] = None
    #print(row.transaction_dt,row.credit_acct, row.amount)
#print(amount)
#print(accounts)

#Are they all external? Yes
#Get vendor ID and accounts
query = S.query(DB.AccountExternal)
for row in query:
    #print(row.account_id)
    if row.account_id in accounts:
        accounts[row.account_id] = (row.iban, row.vendor_id)
        #print("TEST")

print(accounts)


#Get account transactions and sum them.
summed_amounts = 0

for acc in accounts:
    iban = accounts[acc][0]
    sub_sum = 0
    query = S.query(DB.BankTransferFile).filter_by(credit_account_no=iban)
    for row in query:
        amount = float(row.amount)
        print(amount)
        sub_sum += amount
    print(iban+":",sub_sum)
    summed_amounts += sub_sum

print("TOTAL:",summed_amounts)

2017-04-07 18:17:52,425 INFO sqlalchemy.engine.base.Engine SELECT transactions.transaction_id AS transactions_transaction_id, transactions.transaction_ref AS transactions_transaction_ref, transactions.transaction_dt AS transactions_transaction_dt, transactions.amount AS transactions_amount, transactions.debit_acct AS transactions_debit_acct, transactions.credit_acct AS transactions_credit_acct, transactions.status AS transactions_status, transactions.narrative AS transactions_narrative, transactions.approval_dt AS transactions_approval_dt, transactions.approved_by_user_id AS transactions_approved_by_user_id 
FROM transactions 
WHERE transactions.approved_by_user_id = ?
2017-04-07 18:17:52,427 INFO sqlalchemy.engine.base.Engine ('u8284',)
2017-04-07 18:17:52,445 INFO sqlalchemy.engine.base.Engine SELECT account_external.account_id AS account_external_account_id, account_external.vendor_id AS account_external_vendor_id, account_external.effective_dt AS account_external_effective_dt, acco

In [12]:
#Actual amount frauded (includes 100 at start?)

S = DB.sess
query = S.query(DB.Transactions).filter_by(approved_by_user_id="u8284") #account used for fraud.
total_amount = 0

for row in query:
    total_amount += float(row.total)

print(total_amount)

2017-04-07 18:24:44,808 INFO sqlalchemy.engine.base.Engine SELECT invoice_purchase_order.invoice_id AS invoice_purchase_order_invoice_id, invoice_purchase_order.po_id AS invoice_purchase_order_po_id, invoice_purchase_order.vendor_id AS invoice_purchase_order_vendor_id, invoice_purchase_order.invoice_dt AS invoice_purchase_order_invoice_dt, invoice_purchase_order.invoice_image_path AS invoice_purchase_order_invoice_image_path, invoice_purchase_order.payment_due_by_dt AS invoice_purchase_order_payment_due_by_dt, invoice_purchase_order.received_dt AS invoice_purchase_order_received_dt, invoice_purchase_order.received_by_user_id AS invoice_purchase_order_received_by_user_id, invoice_purchase_order.total AS invoice_purchase_order_total 
FROM invoice_purchase_order 
WHERE invoice_purchase_order.received_by_user_id = ?
2017-04-07 18:24:44,809 INFO sqlalchemy.engine.base.Engine ('u8284',)
1015100.0


In [17]:
#Amount of suspicious invoice approvals:
from sqlalchemy import func

S = DB.sess
query = S.query(DB.Transactions).filter_by(approved_by_user_id="u8284")
print(query)

#this is /reaaallly/ lazy
counter = 0
for row in query:
    counter +=1

print(counter)

SELECT transactions.transaction_id AS transactions_transaction_id, transactions.transaction_ref AS transactions_transaction_ref, transactions.transaction_dt AS transactions_transaction_dt, transactions.amount AS transactions_amount, transactions.debit_acct AS transactions_debit_acct, transactions.credit_acct AS transactions_credit_acct, transactions.status AS transactions_status, transactions.narrative AS transactions_narrative, transactions.approval_dt AS transactions_approval_dt, transactions.approved_by_user_id AS transactions_approved_by_user_id 
FROM transactions 
WHERE transactions.approved_by_user_id = ?
2017-04-07 18:48:52,310 INFO sqlalchemy.engine.base.Engine SELECT transactions.transaction_id AS transactions_transaction_id, transactions.transaction_ref AS transactions_transaction_ref, transactions.transaction_dt AS transactions_transaction_dt, transactions.amount AS transactions_amount, transactions.debit_acct AS transactions_debit_acct, transactions.credit_acct AS transacti

In [ ]:
#Amount of invoices: 26
#Based on approvals not at 00:00 all done by u8204
#Found 26 invoices received by u8204
#To actually link them, check equate over invoicest